In [1]:
from selenium import webdriver
from time import sleep
from datetime import datetime, timedelta
import pytz

tz_arg = pytz.timezone("America/Argentina/Buenos_Aires")
tz_rus = pytz.timezone("Europe/Volgograd")

In [2]:
d = webdriver.Chrome()
sleep(2)
d.maximize_window()
d.get('https://www.fifa.com/worldcup/matches/')

In [16]:
class Partido:
    def __init__(self, html):
        
        self.link = html.get_attribute('href')
        
        fecha_aux = html.find_element_by_class_name("fi-mu__info__datetime").text
        fecha_aux = tz_rus.localize(datetime.strptime(fecha_aux, "%d %b %Y - %H:%M Local time"))
        self.fecha = fecha_aux.astimezone(tz_arg)
        
        self.grupo = html.find_element_by_class_name("fi__info__group").text
        self.equipos = [x.text for x in html.find_elements_by_class_name("fi-t__nText ")]
        
        
        try:
            self.marcador = [int(x) for x in html.find_element_by_class_name("fi-s__scoreText").text.split('-')]
            self.pendiente = False
        except:
            self.pendiente = True
        
    def __repr__(self):
        if self.pendiente:
            return "%s [%s] %s - %s" % (self.fecha, self.grupo,
                                          self.equipos[0], self.equipos[1])
        else:
            return "%s [%s] %s %d - %s %d" % (self.fecha, self.grupo,
                                          self.equipos[0], self.marcador[0],
                                          self.equipos[1], self.marcador[1])

In [17]:
fixture = {}

for link in d.find_elements_by_class_name('fi-mu__link'):
    partido = Partido(link)
       
    # Compruebo que el partido esté terminado o en marcha
    if partido.pendiente:
        continue
        
    print(partido)
   
    if not partido.grupo in fixture:
        fixture[partido.grupo] = []
        
    fixture[partido.grupo].append(partido)

2018-06-14 12:00:00-03:00 [Group A] Russia 5 - Saudi Arabia 0
2018-06-15 11:00:00-03:00 [Group A] Egypt 0 - Uruguay 1
2018-06-15 12:00:00-03:00 [Group B] Morocco 0 - IR Iran 1
2018-06-15 15:00:00-03:00 [Group B] Portugal 1 - Spain 0


In [21]:
# Busco los goles
for grupo in fixture.keys():
    for partido in fixture[grupo]:
        d.get(partido.link)
        break
    break

In [ ]:
# Terminados
for html_partido in d.find_elements_by_class_name('result'):
    partido = Partido(html_partido)
    print(partido)
    
# Ongoing
for html_partido in d.find_elements_by_class_name('live'):
    partido = Partido(html_partido)
    print(partido)